In [1]:
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.evaluation import RegressionMetrics


#PreProcessing the data
#raw_data_1 = sc.textFile("/FileStore/tables/g235hl4r1479696958391/ratings.csv")

raw_data_1 = sc.textFile("/FileStore/tables/ods06qe21479752955375/ratings.csv")
header = raw_data_1.take(1)[0]
ratings_data = raw_data_1.filter(lambda l: l!=header).map(lambda l: l.split(",")).map(lambda s: (int(s[0]),int(s[1]),float(s[2]))).filter(lambda s : s[2]!=99).map(lambda s: (s[0],s[1],s[2]/10)).cache()

#splitting the dataset to test and train
#train1,test1 = ratings_data.randomSplit([0.9,0.1])
train,test=ratings_data.randomSplit([0.8,0.2])
test_predict=test.map(lambda s:(s[0],s[1]))
test=test.map(lambda s:((s[0], s[1]), s[2]))


# Building the model
l_array= [0.001,0.005,0.05,0.5]
r_array = [2,4,6,8,12,18]

#l_array=[0.001]
#r_array=[2]
min_rmse = float('inf')

for l in l_array:
    for r in r_array:
        model = ALS.train(train, rank=r,iterations=10,lambda_=l, blocks=10,nonnegative=True,seed=None)
        predict_obs = model.predictAll(test_predict).map(lambda s: ((s[0], s[1]), s[2])).join(test).map(lambda (k,(v1,v2)):(v1,v2))
        #print(predict_obs.take(1000))
        metrics = RegressionMetrics(predict_obs)
        abs_error=metrics.meanAbsoluteError
        rmse=metrics.rootMeanSquaredError
        variance=metrics.explainedVariance
        sq_error=metrics.meanSquaredError
        print 'Model with rank=%s and lambda=%s has rmse=%s; abs_error=%s; variance=%s; squared_error=%s ' % (r,l,rmse,abs_error,variance,sq_error)
        if rmse < min_rmse:
            min_rmse = rmse
            best_rank = r
            best_l =l
            best_metrics=metrics
            best_model=model

print(min_rmse)            
abs_error=best_metrics.meanAbsoluteError
rmse=best_metrics.rootMeanSquaredError
variance=best_metrics.explainedVariance
sq_error=best_metrics.meanSquaredError
        
print 'The best model was trained with rank %s and lambda=%s' % (best_rank,best_l)
print 'This model has metrics as follows'
print 'root mean square error=%s' % rmse
print 'mean absolute error=%s' % abs_error
print 'variance=%s' % variance
print ''

#recommends top 5 jokes(by joke id) for each user id
recommended_products=model.recommendProductsForUsers(5).map(lambda l:(l[0],(l[1][0][1],l[1][1][1],l[1][2][1],l[1][3][1],l[1][4][1])))
#displaying only first 100 results
print 'displaying 100 results of 5 jokes recommended for each user id'
print recommended_products.take(100)


print ''

#recommends top 5 users who are mostly likely going to like the joke for all the jokes
recommeded_users=model.recommendUsersForProducts(5).map(lambda l:(l[0],(l[1][0][0],l[1][1][0],l[1][2][0],l[1][3][0],l[1][4][0])))
#displaying only first 100 results
print 'displaying 100 results of 5 users recommended for each joke id'
print recommeded_users.take(100)
